## 1. Install 

In [4]:
!pip install transformers

     |████████████████████████████████| 660 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 30.7 MB/s eta 0:00:01
     |████████████████████████████████| 675 kB 78.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 107.8 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893259 sha256=dfdb3527161ee733bb40f878d8f119e1bf1a94a43d6add973d68e6266397bfaa
  Stored in directory: /home/ubuntu/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses


In [2]:
question="What is the best method to combat the hypercoagulable state seen in COVID-19 ?"
file_name="sm_df.txt"
non_answer='[CLS] what is the best method to combat the hypercoagulable state seen in covid - 19 ? [SEP]'

In [5]:
#from mrjob.job import MRJob
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
import pandas as pd
import json 

In [6]:
data_output=[]
with open(file_name, 'r') as f:
    for line in f:
        #if len(data_output)>3:
        #    break
        con=json.loads(line)
        paper_id=con["paper_id"]
        text_dict=con["text_dict"]
        for key in text_dict.keys():
            content = text_dict[key]
            com_id=str(paper_id)+"__"+str(key)

            answer_text=str(content)        

            input_ids = tokenizer.encode(question, answer_text,max_length=500)

            # Report how long the input sequence is.
            #print('Query has {:,} tokens.\n'.format(len(input_ids)))

            # ======== Set Segment IDs ========
            # Search the input_ids for the first instance of the `[SEP]` token.
            sep_index = input_ids.index(tokenizer.sep_token_id)

            # The number of segment A tokens includes the [SEP] token istelf.
            num_seg_a = sep_index + 1

            # The remainder are segment B.
            num_seg_b = len(input_ids) - num_seg_a

            # Construct the list of 0s and 1s.
            segment_ids = [0]*num_seg_a + [1]*num_seg_b

            # There should be a segment_id for every input token.
            assert len(segment_ids) == len(input_ids)

            # ======== Evaluate ========
            # Run our example question through the model.
            start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                            token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

            # ======== Reconstruct Answer ========
            # Find the tokens with the highest `start` and `end` scores.
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)


            # Get the string versions of the input tokens.
            tokens = tokenizer.convert_ids_to_tokens(input_ids)

            # Start with the first token.
            answer = tokens[answer_start]

            # Select the remaining answer tokens and join them with whitespace.
            for i in range(answer_start + 1, answer_end + 1):

                # If it's a subword token, then recombine it with the previous token.
                if tokens[i][0:2] == '##':
                    answer += tokens[i][2:]

                # Otherwise, add a space then the token.
                else:
                    answer += ' ' + tokens[i]

            s_scores = start_scores.detach().numpy().flatten()
            e_scores = end_scores.detach().numpy().flatten()  
            print(com_id)
            if (answer!="[CLS]") & (float(max(s_scores))>0)& (len(answer)>0)&(answer!=non_answer):
                data_output.append([paper_id,key,answer,max(s_scores)])

1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__0_
1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__1_Purpose
1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__2_Description
1d7234df1a19a0cf0b4038ceb31e7a97881cdbd7__3_DISCUSSION
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__0_Background
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__1_Selection of potential HLA-A*0201 binding peptides derived from SARS-CoV/S protein
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__2_Ex vivo generation of peptide-specific CD8 + CTLs from healthy human donor PBLs
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__3_In vivo induction of peptide-specific CD8 + CTLs in HLA-A2.1/K b transgenic mice
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__4_Discussion
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__5_Conclusion
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__6_Peptides
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__7_Cells and Cell Culture
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__8_Animals
0ced1f946cce007aa319a0ba38aef2c4b14dab0e__9_Generation of CTLs in healthy donor

KeyboardInterrupt: 

In [22]:
data_output[3]

['0ced1f946cce007aa319a0ba38aef2c4b14dab0e',
 '0_Background',
 'neutralizing antibodies',
 1.7667923]

In [23]:
# load additional module
import pickle

# define a list of places
with open('output_sample.data', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(data_output, filehandle)

In [24]:
with open('output_sample.data', 'rb') as filehandle:
    # read the data as binary data stream
    placesList = pickle.load(filehandle)

In [ ]:
for i in data_output:
    print()

In [7]:
len(data_output)

2284

In [8]:
paper_id_list=[i[0] for i in data_output]

In [11]:
len(set(paper_id_list))

441

In [16]:
[print(i[2]) for i in data_output]

simulation
educate
video recording and sharing
neutralizing antibodies
im
initiatives and response mechanisms
[CLS] what is the best method to combat the hypercoagulable state seen in covid - 19 ? [SEP] rinderpest is german for " cattle plague " . it is a highly infectious viral disease that can affect domesticated and wild , cloven - hoofed mammals . african cattle and buffalo are extremely susceptible to the disease while sheep and goats experience a mild form that is not as acute , resulting in lower fatalities . in susceptible animal , rinderpest infection has a fatality rate greater than 90 % and a mortality rate of 100 % . in the 1890 ' s , an outbreak of rinderpest south of the river zambezi killed about 5 . 2 million cattle , sheep , goats and oxen , and an unknown number of wild giraffe , buffalo and wildebeest . this rinderpest epidemic , happening at a time of much historical tumult , was the most devastating epidemic to hit southern africa in the late nineteenth century . t

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
paper_id_list[-1]

'3ae24898796b7972372f6ed11be3d07c44e8cfd5'

In [18]:
data_paper_id=[]
n=0
with open(file_name, 'r') as f:
    for line in f:
        con=json.loads(line)
        n=n+1
        paper_id=con["paper_id"]
        data_paper_id.append(paper_id)
        if paper_id == '3ae24898796b7972372f6ed11be3d07c44e8cfd5':
            print(n)
            break

520


In [19]:
len(data_paper_id)

520

In [20]:
len(set(data_paper_id))

520